# **Instrucciones:**
En la hoja “ax” se encuentra el cálculo de las anualidades cuyo resultado se encuentra en la celda “d4”.
El objetivo es llenar la hoja “Matrizax”, con el resultado expresado en la hoja “ax”, considerando las variables de año y edad.  
Las edades que hay que considerar es de 60 a 111 años y los años de 2023 a 2133. La muestra está en la hoja “Matrizax”.

#Importamos Datos

In [ ]:
#Instalamos librería para manipular archivos Excel
!pip install openpyxl

In [ ]:
#Importamos librerías
import openpyxl
import pandas as pd
import numpy as np

In [ ]:
#Definimos función que transforma un rango dado en un df
def load_workbook_range(range_string, ws):
    data_rows = []
    for row in ws[range_string]:
        data_rows.append([cell.value for cell in row])

    return pd.DataFrame(data_rows, )

#Definimos función que registra el último renglón y columna con valores de una hoja de excel
def cell_max_sheet(ws): #se podria hacer setiando el atributo y con una función recursiva pero no deja :(
  max_row = ws.max_row
  max_col = ws.max_column
  while ws.cell(max_row,1).value is None :
    max_row -=  1
  while ws.cell(1,max_col).value is None :
    max_col -=  1
  max_col = chr(ord('@')+max_col) #Convertimos número a letra
  max_cell = 'A1:' + max_col + str(max_row)

  return max_cell

## Manipulamos los Datos

In [ ]:
#Cargamos archivo de excel
wb = openpyxl.load_workbook(filename="Anualidades.xlsx", data_only=True)
#Cargamos hojas de las tablas
Mortality_sheet = wb['Tablas']
Interest_sheet = wb['Tasas']

#Cargamos tablas de datos a usar
Mortality_table = load_workbook_range(cell_max_sheet(Mortality_sheet), Mortality_sheet)
Interest_table = load_workbook_range(cell_max_sheet(Interest_sheet), Interest_sheet)

#Limpiamos la tabla y renombramos títulos
Mortality_table = Mortality_table.rename(columns=Mortality_table.iloc[0]).drop(Mortality_table.index[0]).dropna(axis = 1, how='all')
Mortality_table.at[112, ('Mejora H', 'Mejora m')] = 0 #Seteamos ultimos valores de mejora a 0
Interest_table = Interest_table.rename(columns=Interest_table.iloc[0]).drop(Interest_table.index[0])

##Cálculo de la matriz con anualidades

In [ ]:
#Funciones auxiliares para calcular la anualidad dado una edad de inicio fija y un año fijo
def annuity_x_y(age, year, titular, gto_adm, mortality_table, interest_table):
    year_fixed = np.arange(year, year + 111)  # Arreglo de años a considerar
    age_fixed = np.arange(age, age + 111)  # Arreglo de edades a considerar
    tasa_dinamica = Tasa_dinamica(year_fixed, age_fixed, titular, mortality_table)
    vk_values = vk(year_fixed, interest_table)
    annuity = np.sum(kpx(tasa_dinamica) * vk_values)
    annuity_12 = ((annuity - 11 / 24) * 12) * (1 + gto_adm)  # Mensualizamos la anualidad
    return annuity_12

def Tasa_dinamica(year_fixed, age_fixed, titular, mortality_table):
    # Se cambia la tabla de mortalidad dependiendo la condición
    if titular == 'CMG':
        lst_mort = mortality_table.loc[mortality_table['Edad'].isin(age_fixed), 'EMSSAH-RCS-15 ']
    else:
        lst_mort = mortality_table.loc[mortality_table['Edad'].isin(age_fixed), 'RJP H']

    # Calculamos mejora y guardamos en una lista de longitud 111
    lst_mejora = mortality_table.loc[mortality_table['Edad'].isin(age_fixed), 'Mejora H']
    year_from22 = np.arange(year_fixed[0] - 2022, year_fixed[0] - 2022 + len(lst_mort))

    # Aplicamos mejora
    lst_final = (np.power(1 - lst_mejora, [min(100, x) for x in year_from22]) * lst_mort).tolist()

    # Agregamos 1's si la edad sobrepasa la de la tabla de mortalidad
    lst_final.extend([1] * (111 - len(lst_final)))
    return lst_final

def kpx(tasa_din):
    list_kpx = [1, 2 * (1 - tasa_din[0]) / (1 + (1 - tasa_din[0]))]
    for i in range(1, len(tasa_din) - 1):
        list_kpx.append(list_kpx[i] * (1 - tasa_din[i]))
    return np.array(list_kpx) #Muy importante siempre usar np.arrays para optimizar tiempos de lectura de memoria

def vk(year_fixed, interest_table):
    list_vk = [1]
    t = np.arange(len(year_fixed))
    interest_fixed = interest_table.loc[interest_table['Año'].isin(year_fixed), 'Esc Riesgo'].tolist()
    for i in range(1, len(interest_fixed)):
        list_vk.append((1 + interest_fixed[i]) ** (-t[i]) * (1 + interest_fixed[1]) ** (1 / 2))
    return np.array(list_vk)

## Método principal que obtiene la matrix de anualidades

In [ ]:
#Obtenemos último año y edad para generar matriz
last_age = Mortality_table['Edad'].iloc[-1]
last_year = 2023 + last_age - 1

#Valores fijos
Titular = 'CMG'
Gto_adm = 0.02

# Creamos arreglos de edad y año
ages = np.arange(60, last_age + 1)
years = np.arange(2023, last_year + 1)

# Generamos matriz con las anualidades
matrix = np.zeros((len(ages), len(years)))

#Se podría optimizar con broadcasting pero es algo complicado
for i, age in enumerate(ages) :
  for j, year in enumerate(years) :
    matrix[i, j] = annuity_x_y(age, year, Titular, Gto_adm, Mortality_table, Interest_table)

## Creación de DataFrame para visualización y carga a excel

In [ ]:
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import files

#Conversión a dataframe
df = pd.DataFrame(matrix, columns = np.arange(2023, last_year + 1), index = np.arange(60, last_age + 1))
df.head()

#Cargamos libro de excel
wb = openpyxl.load_workbook(filename="Anualidades.xlsx", data_only=True)
#Cargamos en una nueva pestaña del archivo de excel los resultados
if 'Matrixax_Python' in wb.sheetnames:
  ws = wb['Matrixax_Python']
else:
  wb.create_sheet('Matrixax_Python')
  ws = wb['Matrixax_Python']

#Desarrollamos df
if ws.cell(2,1).value is None :
  for r in dataframe_to_rows(df, index=True, header=True):
      ws.append(r)
  for i in (1,1,2): ws.delete_rows(i,1)

#Guarda y descarga automáticamente archivo
wb.save('Anualidadesv2.xlsx')
files.download('Anualidadesv2.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import pandas as pd

# Cargamos archivo de Excel
wb = pd.read_excel("Anualidades.xlsx", sheet_name=None)

# Cargamos tablas de datos
Mortality_table = wb['Tablas'].rename(columns=wb['Tablas'].iloc[0]).drop(wb['Tablas'].index[0]).dropna(axis=1, how='all')
Mortality_table.at[112, ('Mejora H', 'Mejora m')] = 0  # Seteamos últimos valores de mejora a 0
Interest_table = wb['Tasas'].rename(columns=wb['Tasas'].iloc[0]).drop(wb['Tasas'].index[0])

# Función para calcular la anualidad
def annuity_x_y(age, year, titular, gto_adm, mortality_table, interest_table):
    year_fixed = np.arange(year, year + 111)
    age_fixed = np.arange(age, age + 111)
    tasa_dinamica = Tasa_dinamica(year_fixed, age_fixed, titular, mortality_table)
    vk_values = vk(year_fixed, interest_table)
    annuity = np.sum(kpx(tasa_dinamica) * vk_values)
    annuity_12 = ((annuity - 11 / 24) * 12) * (1 + gto_adm)  # Mensualizamos la anualidad
    return annuity_12

@np.vectorize
def Tasa_dinamica(year_fixed, age_fixed, titular, mortality_table):
    if titular == 'CMG':
        lst_mort = mortality_table.loc[mortality_table['Edad'].isin(age_fixed), 'EMSSAH-RCS-15 ']
    else:
        lst_mort = mortality_table.loc[mortality_table['Edad'].isin(age_fixed), 'RJP H']

    lst_mejora = mortality_table.loc[mortality_table['Edad'].isin(age_fixed), 'Mejora H']
    year_from22 = np.arange(year_fixed - 2022, year_fixed - 2022 + len(lst_mort))
    lst_final = (np.power(1 - lst_mejora, np.minimum(100, year_from22)) * lst_mort).tolist()
    lst_final.extend([1] * (111 - len(lst_final)))
    return lst_final

def kpx(tasa_din):
    kpx_values = np.cumprod(1 - tasa_din)
    kpx_values[0] = 1
    kpx_values[1] = 2 * (1 - tasa_din[0]) / (1 + (1 - tasa_din[0]))
    return kpx_values

def vk(year_fixed, interest_table):
    t = np.arange(len(year_fixed))
    interest_fixed = interest_table.loc[interest_table['Año'].isin(year_fixed), 'Esc Riesgo'].values
    vk_values = np.cumprod((1 + interest_fixed) ** (-t))
    vk_values[0] = 1
    vk_values[1:] *= (1 + interest_fixed[1]) ** (1 / 2)
    return vk_values

# Obtenemos último año y edad para generar matriz
last_age = Mortality_table['Edad'].iloc[-1]
last_year = 2023 + last_age - 1

# Valores fijos
Titular = 'CMG'
Gto_adm = 0.02

# Creamos arreglos de edad y año
ages = np.arange(60, last_age + 1)
years = np.arange(2023, last_year + 1)

# Generamos matriz con las anualidades
matrix = np.zeros((len(ages), len(years)))
for i, age in enumerate(ages):
    for j, year in enumerate(years):
        matrix[i, j] = annuity_x_y(age, year, Titular, Gto_adm, Mortality_table, Interest_table)